## Creating Neural Network Model for Hand Gesture Classification 

Gestures for 

*  Play
*   Pause




In [ ]:
#Loading Dataset

!git clone https://github.com/anju-suresh/dataset.git

fatal: destination path 'dataset' already exists and is not an empty directory.


In [ ]:
#Importing libraries

import numpy as np
from imutils import paths
import os

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

2023-01-06 11:01:05.298444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#Finding Image path

imagePaths = list(paths.list_images('dataset'))

In [ ]:
#Retrieving Labels from Image Paths

data = []
labels = []

IMG_SIZE = 224
CHANNELS = 3

for imagePath in imagePaths:
  label = imagePath.split(os.path.sep)[-2]
  image = load_img(imagePath, target_size=(IMG_SIZE, IMG_SIZE))
  image = img_to_array(image)
  image = image/255

  data.append(image)
  labels.append(label)


data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
#Finding Label Values

np. unique(labels, return_counts=True)

(array(['play', 'stop'], dtype='<U4'), array([300, 300]))

### Preprocessing the label by encoding them

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels)

In [ ]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split
(trainX, testX, trainY, testY) = train_test_split(data, labels,	test_size=0.20, stratify=labels, random_state=42)

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input

In [ ]:
#Using ResNet for Feature Extraction

feature_extractor_layer = ResNet50V2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(IMG_SIZE,IMG_SIZE,CHANNELS)))

2023-01-06 11:01:13.279183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
feature_extractor_layer.trainable = False

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [ ]:
# Model building

model = tf.keras.Sequential()
model.add(feature_extractor_layer)
model.add(layers.Flatten(name="flatten"))
model.add(layers.Dense(1024, activation='relu', name='hidden_layer'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax', name='output'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 hidden_layer (Dense)        (None, 1024)              102761472 
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 output (Dense)              (None, 2)                 2050      
                                                                 
Total params: 126,328,322
Trainable params: 102,763,522
Non-trainable params: 23,564,800
_________________________________________________________________


In [ ]:
#Compliling Model

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
  loss="categorical_crossentropy",
  metrics=["accuracy"])

In [ ]:
#Pre-processing Image before training Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
#Fitting Model

history = model.fit(aug.flow(trainX, trainY),
	                  validation_data=(testX, testY),
	                  epochs=10)

Epoch 1/10
15/15 [==============================] - 45s 3s/step - loss: 0.2297 - accuracy: 0.9271 - val_loss: 9.9341e-10 - val_accuracy: 1.0000
Epoch 2/10
15/15 [==============================] - 40s 3s/step - loss: 2.6000e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
15/15 [==============================] - 43s 3s/step - loss: 1.6069e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
15/15 [==============================] - 40s 3s/step - loss: 2.0163e-05 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
15/15 [==============================] - 41s 3s/step - loss: 6.0868e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
15/15 [==============================] - 41s 3s/step - loss: 2.4764e-06 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
15/15 [==============================] - 46s 3s/step - loss: 4.0313e-05 - accuracy: 1.0000 - val_loss: 0.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
predIdxs = model.predict(testX)

4/4 [==============================] - 8s 2s/step


In [ ]:
predIdxs[0]

array([2.4363586e-12, 1.0000000e+00], dtype=float32)

In [ ]:
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=le.classes_))

              precision    recall  f1-score   support

        play       1.00      1.00      1.00        60
        stop       1.00      1.00      1.00        60

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120



In [ ]:
model.save('model_hand_gesture.h5', save_format="h5")

1.2G	model_hand_gesture.h5
